In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
wavelength = []
intensity =[]
with open("slit9_of_Baldwin_et_al_1991_corrpos_sum_nall.dat","r") as f:
    for line in f.readlines():
        line = line.split(' ')
        wavelength.append(float(line[0]))
        intensity.append(float(line[1]))
        
wavelength = np.array(wavelength)
intensity = np.array(intensity)

In [3]:
plot(wavelength,intensity)

In [8]:
import pyfits
image = np.copy(pyfits.open("emhalpha_flux.fits")[0].data)

In [9]:
imshow(image)

In [7]:
plot(intensity)

In [91]:
def thresh(y,y0):
    """
    Find all intervals above a threshold
    """
    intervals = []
    idx = []
    
    a = None
    high = False
    
    for i,yy in enumerate(y):
        
        if yy >= y0:            
            if high == False:
                a = i
                high = True
        else:
            if high == True:
                intervals.append(y[a:i])
                idx.append(a)
                a = None
            high = False
            
    return np.array(idx), np.array(intervals)
             

class tree(object):
    
    def __init__(self,y,offset,nlevels,accum='root'):
        
        if accum == 'root':
            accum = np.zeros(len(y))
        
        
        
        self.accum = accum
        self.y = y
        self.offset = offset
        self.max = max(y)
        self.argmax = np.argmax(y)+offset
        
        print self.argmax
        
        count[self.argmax] += 1
        
        if (nlevels == 0):
            self.subtrees = None
            return
        
        idx, intervals = thresh(y,levels[0])
        idx += offset
        
        self.idx = idx
        self.intervals = intervals
        
        self.subtrees = []
        
        for _idx, _interval in zip(idx,intervals):
            
            self.subtrees.append(tree(_interval,_idx,levels[1:],count))
        


In [33]:
h,b = np.histogram(image,bins=2000)
b = b[:-1] + .5*np.diff(b)
dflux = b * h
ndflux = dflux/np.sum(dflux)

In [21]:
plot(b,dflux/np.sum(dflux))

In [34]:
plot(b,np.cumsum(ndflux),marker='+')

In [26]:
b

array([  0.00000000e+00,   2.95281773e+04,   5.90563545e+04, ...,
         5.89972982e+07,   5.90268264e+07,   5.90563545e+07])

In [29]:
a=.1
w=.1*a
b[(ndflux < a+w) and (ndflux > a-w)]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [43]:
perimeter_bins = b[np.logical_and( (np.cumsum(ndflux) > a-w) , ((np.cumsum(ndflux) < a+w)) )]
low = perimeter_bins[0]
high = perimeter_bins[-1]

In [45]:
image.flatten()